In [13]:
import sys

sys.path.append("../")

# load from local .env file
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv(), override=True)

# external files
from src.preprocessor.preprocessing import FileIO
from src.database.weaviate_interface_v4 import WeaviateIndexer, WeaviateWCS
from src.database.database_utils import get_weaviate_client

# standards
import os
import time
import json
from typing import List
from tqdm import tqdm
from rich import (
    print,
)  # nice library that provides improved printing output (overrides default print function)

from src.database.properties_template import properties

In [14]:
# read env vars from local .env file
api_key = os.environ["WEAVIATE_API_KEY"]
url = os.environ["WEAVIATE_ENDPOINT"]

model_names = [
    "../models/bge-base-finetuned-500",
]

base = ["bge_finetuned_500"]

chunk_sizes = [128, 256, 512]

for chunk_size in chunk_sizes:
    for model_path, bas in zip(model_names, base):
        # instantiate client,
        client = WeaviateWCS(
            endpoint=url, api_key=api_key, model_name_or_path=model_path
        )
        print(client._client.is_live())

        # Alternatively you can simply use the get_weaviate_client
        # convenice function, which assumes a default config
        # client = get_weaviate_client()

        data_path = f"../data/huberman-{bas}-{chunk_size}.parquet"

        data = FileIO.load_parquet(data_path)

        collection_name = f"huberman_{bas}_{chunk_size}"

        client.create_collection(
            collection_name=collection_name,
            properties=properties,
            description="Huberman Labs: 193 full-length transcripts",
        )

        indexer = WeaviateIndexer(client)

        batch_object = indexer.batch_index_data(data, collection_name)

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.2.2. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



/var/folders/m5/q3xxb9ld23z910ts0qsq57lw0000gn/T/ipykernel_28254/2456306240.py:18: ResourceWarning: unclosed <ssl.SSLSocket fd=77, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=0, laddr=('192.168.0.28', 62872), raddr=('34.149.137.116', 443)>
  client = WeaviateWCS(


True

Shape of data: (51128, 13)
Memory Usage: 5.07+ MB
Error creating collection, due to: Collection may not have been created properly.! Unexpected status code: 422, with response body: {'error': [{'message': "'Huberman_qa-mpnet-base-dot-v1_128' is not a valid class name"}]}.
Collection "huberman_qa-mpnet-base-dot-v1_128" not found on host, creating Collection first...


ValueError: Tried to create Collection <huberman_qa-mpnet-base-dot-v1_128> but no properties were provided.

In [15]:
print(properties)

[
    Property(
        name='video_id',
        dataType=<DataType.TEXT: 'text'>,
        description='Unique identifier of the video episode',
        indexFilterable=True,
        indexSearchable=False,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='title',
        dataType=<DataType.TEXT: 'text'>,
        description='Title of the video episode',
        indexFilterable=True,
        indexSearchable=True,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='length_seconds',
        dataType=<DataType.INT: 'int'>,
        description='Length of the video episode in seconds',
        indexFilterable=True,
        indexSearchable=False,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='thumbnail_url',
        dataType=<DataType.TEXT: 'text'>,
        description='URL of the video episode thumbnail picture',
        indexFilterable=False,
        indexSearchable=False,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='keywords',
        dataType=<DataType.TEXT_ARRAY: 'text[]'>,
        description='Keywords associated with the video episode',
        indexFilterable=True,
        indexSearchable=True,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='summary',
        dataType=<DataType.TEXT: 'text'>,
        description='Short summary of the episode content',
        indexFilterable=False,
        indexSearchable=True,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='view_count',
        dataType=<DataType.INT: 'int'>,
        description='Number of views the video episode has received',
        indexFilterable=True,
        indexSearchable=False,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='episode_num',
        dataType=<DataType.INT: 'int'>,
        description='User-defined episode number in the series',
        indexFilterable=True,
        indexSearchable=False,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='doc_id',
        dataType=<DataType.TEXT: 'text'>,
        description='Unique document identifier created during preprocessing of documents',
        indexFilterable=True,
        indexSearchable=False,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='content',
        dataType=<DataType.TEXT: 'text'>,
        description='Chunk of text from the video episode',
        indexFilterable=False,
        indexSearchable=True,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='guest',
        dataType=<DataType.TEXT: 'text'>,
        description='Guests featured in the video episode',
        indexFilterable=True,
        indexSearchable=True,
        nestedProperties=None,
        skip_vectorization=False,
        tokenization=None,
        vectorize_property_name=True
    ),
    Property(
        name='episode_url',
        dataType=<DataType.TEXT: 'text'>,
        description='URL of the video episode on YouTube',
        indexFilterable=False,
        indexSearchable=False,
        nestedPr